In [ ]:
import requests # Helps construct the request to send to the API
import json # JSON helper functions
from bs4 import BeautifulSoup

In [ ]:
#get data from craigslist

url = 'http://washingtondc.craigslist.org/nva/apa/4964060272.html'

# Make the request
response = requests.post(url)

# Confirm the response worked
response.status_code

In [ ]:
soup = BeautifulSoup(response.text)
print(soup.prettify())

In [ ]:
#get attributes about property - preset choices from craigslist
attribute_list = {}

x = soup.find('p',class_='attrgroup').find_all('span')
for attribute in x:

    if 'BR' in attribute.text:
        attribute_list['bedroom'] = attribute.text.split('/')[0].replace('BR','')
    if 'Ba' in attribute.text:
        attribute_list['bathroom'] = attribute.text.split('/')[1].replace('Ba','')
    if 'ft' in attribute.text:
        attribute_list['square_footage'] = attribute.text.replace('ft2','')
    if attribute.text in ['apartment', 'condo', 'cottage/cabin', 'duplex', 'flat', 
        'house', 'in-law', 'loft', 'townhouse','manufactured', 'assisted living', 'land']:
        attribute_list['housing_type'] = attribute.text
    if 'cat' in attribute.text:
        attribute_list['cat'] = attribute.text
    if 'dog' in attribute.text:
        attribute_list['dog'] = attribute.text
    if attribute.text in ['w/d in unit','laundry in bldg','laundry on site','w/d hookups']:
        attribute_list['laundry'] = attribute.text 
    if attribute.text in ['carport', 'attached garage', 'detached garage',  'off-street parking', 'street parking', 'valet parking']:
        attribute_list['parking'] = attribute.text
    if 'smoking' in attribute.text:
        attribute_list['smoking'] = attribute.text

attribute_list['availability'] = soup.find('span', class_='housing_movein_now property_date')['today_msg']
attribute_list['data_available'] = soup.find('span', class_='housing_movein_now property_date')['date']

attribute_list

In [ ]:
soup.find('title')

In [ ]:
x = soup.find_all('h2')
for i in x:
    print i

In [ ]:
#get the listing price
x = soup.find('h2')
print x.contents[3].text

In [ ]:
#get the number of bedrooms
x = soup.find('h2')
print x.contents[5].text.split('-')[0].replace('/','')

In [ ]:
#get the square footage of the listing
x = soup.find('h2')
print x.contents[5].text.split('-')[1][0:-2] #remove extra characters at start of string and '2' at end for square ft

In [ ]:
#Create a list of dicts containing image number, image link, and image size
image_data = []

images = soup.find('figure').find_all('div')
for pic in images[-1]:    
    pic_dict = {}
    pic_dict['image_number'] = pic['title']
    pic_dict['href'] = pic['href']
    pic_dict['image_size'] = pic['href'].split('_')[-1]
    
    image_data.append(pic_dict)

image_data

In [ ]:
#create a dict with  location data: country, state, city, address, longitude, latitude, and location accuracy metric

#get longitude, latitude, and location accuracy metric
location = soup.find('div', class_='viewposting')
location_data_accuracy = soup.find('div', class_='viewposting')['data-accuracy']
latitude = soup.find('div', class_='viewposting')['data-latitude']
longitude = soup.find('div', class_='viewposting')['data-longitude']

#get country, state, city
map = soup.find('p', class_='mapaddress').find_all('a')[1]['href'] #get yahoo maps link - easier to extract data than google maps
country = map.split('country=')[1]
state = map.split('csz=')[1].split('&')[0].split('+')[1]
city  = map.split('csz=')[1].split('&')[0].split('+')[0]

#get address
address = soup.find('div', class_='mapaddress').text

#place all location data into dict
location_dict = {}
location_dict['location_data_accuracy'] = location_data_accuracy
location_dict['latitude'] = latitude
location_dict['longitude'] = longitude
location_dict['country'] = country
location_dict['state'] = state
location_dict['city'] = city
location_dict['address'] = address

location_dict

In [ ]:
#get the written user description of the listing
listing_description = soup.find('section', class_='userbody').find('section')

print listing_description.text